# Benchmarking different quantum compilers


## Load benchmarks

In [36]:
from pathlib import Path

benchmark_files = list(Path(".").rglob("*.qasm"))

## Prepare compilers

In [4]:
compilers = dict()

## QisKit

In [5]:
from qiskit.compiler import transpile

In [76]:
def optimize_qiskit(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    opt_qc = transpile(qc, optimization_level=3, approximation_degree=1.0,
                       basis_gates=['rz', 'rx', 'h', 't', 'tdag', 's', 'cx'])
    return opt_qc.qasm()


compilers['qiskit'] = optimize_qiskit

## PyZX

In [7]:
import pyzx as zx

In [14]:
def optimize_pyzx(qasm):
    c = zx.Circuit.from_qasm(qasm)
    zx.draw(c)
    c = c.to_graph()
    zx.simplify.full_reduce(c)
    c.normalize()
    c_opt = zx.extract_circuit(c.copy(), quiet=True, optimize_cnots=3).to_basic_gates()
    return c_opt.to_qasm()


compilers['pyzx'] = optimize_pyzx

## VOQC

To install voqc run
```shell
git submodule update --remote
cd pyvoqc
opam pin voqc https://github.com/inQWIRE/mlvoqc.git#mapping
./install.sh
```
Then copy pyvoqc/lib to your site packages pyvoqc folder

In [9]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

In [10]:
def optimize_voqc(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    vpm = voqc_pass_manager(post_opts=["optimize"])
    qc_opt = vpm.run(qc)
    return qc_opt.qasm()


compilers['voqc'] = optimize_voqc

## $\mid$tket$\rangle$

In [95]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import SequencePass, RemoveRedundancies, FullPeepholeOptimise, auto_rebase_pass
from pytket.predicates import CompilationUnit
from pytket import OpType

In [98]:
def optimize_tket(qasm):
    qc = circuit_from_qasm_str(qasm)
    gates = {OpType.Rz, OpType.Rx, OpType.H, OpType.T, OpType.Tdg, OpType.S, OpType.CX}
    rebase = auto_rebase_pass(gates)
    seqpass = SequencePass(
        [
            FullPeepholeOptimise(),
            RemoveRedundancies(),
            rebase,
        ])
    cu = CompilationUnit(qc)
    seqpass.apply(cu)
    qc_opt = cu.circuit
    return circuit_to_qasm_str(qc_opt)


compilers['tket'] = optimize_tket

## Staq

In [58]:
import pystaq


In [64]:
def optimize_staq(qasm):
    qc = pystaq.parse_str(qasm)
    pystaq.simplify(qc)
    pystaq.rotation_fold(qc)
    pystaq.simplify(qc)
    pystaq.cnot_resynth(qc)
    pystaq.simplify(qc)
    return str(qc)


compilers['staq'] = optimize_staq


# Run Benchmarks

In [99]:
# Test

qc = QuantumCircuit(2)
qc.rz(0.132312, 0)
qc.t(0)
qc.h(1)
qc.cnot(0, 1)
qc.t(0)
qc.cnot(0, 1)
qc.h(0)
qc.t(0)
qc.t(1)
qc.s(1)
qc.cnot(1, 0)

print("Original")
print(qc.draw())

qasm = qc.qasm()

for compiler_name in compilers:
    print(compiler_name)
    qasm_opt = compilers[compiler_name](qasm)
    qc_opt = QuantumCircuit.from_qasm_str(qasm_opt)
    print(qc_opt.draw())


Original
     ┌─────────────┐┌───┐     ┌───┐     ┌───┐┌───┐┌───┐
q_0: ┤ Rz(0.13231) ├┤ T ├──■──┤ T ├──■──┤ H ├┤ T ├┤ X ├
     └────┬───┬────┘└───┘┌─┴─┐└───┘┌─┴─┐├───┤├───┤└─┬─┘
q_1: ─────┤ H ├──────────┤ X ├─────┤ X ├┤ T ├┤ S ├──■──
          └───┘          └───┘     └───┘└───┘└───┘     
qiskit
     ┌─────────────┐┌──────────┐┌──────────┐     ┌─────────┐┌──────────┐»
q_0: ┤ Rz(-1.4385) ├┤ Rx(3π/4) ├┤ Rz(-π/2) ├──■──┤ Rx(π/2) ├┤ Rz(-π/2) ├»
     ├─────────────┤└──────────┘└──────────┘┌─┴─┐├─────────┤├─────────┬┘»
q_1: ┤ Rx(-3.0747) ├────────────────────────┤ X ├┤ Rz(π/2) ├┤ Rx(π/2) ├─»
     └─────────────┘                        └───┘└─────────┘└─────────┘ »
«                    
«q_0: ───────────────
«     ┌─────────────┐
«q_1: ┤ Rz(-2.4231) ├
«     └─────────────┘
pyzx


     ┌────────────┐┌───┐┌─────────┐┌───┐      ┌───┐              
q_0: ┤ Rz(1.7031) ├┤ H ├┤ Rz(π/4) ├┤ H ├─■────┤ H ├──────────────
     └───┬───┬────┘└───┘└─────────┘└───┘ │ ┌──┴───┴───┐┌───┐┌───┐
q_1: ────┤ H ├───────────────────────────■─┤ Rz(3π/4) ├┤ H ├┤ H ├
         └───┘                             └──────────┘└───┘└───┘
voqc
     ┌────────────┐   ┌───┐    ┌───┐┌───┐
q_0: ┤ Rz(1.7031) ├───┤ H ├────┤ T ├┤ X ├
     └───┬───┬────┘┌──┴───┴───┐└───┘└─┬─┘
q_1: ────┤ H ├─────┤ Rz(3π/4) ├───────■──
         └───┘     └──────────┘          
tket
     ┌────────────┐┌─────────┐ ┌──────────┐┌───┐
q_0: ┤ Rz(3.2739) ├┤ Rx(π/2) ├─┤ Rz(3π/4) ├┤ X ├
     └┬──────────┬┘├─────────┴┐├──────────┤└─┬─┘
q_1: ─┤ Rz(7π/2) ├─┤ Rx(7π/2) ├┤ Rz(9π/4) ├──■──
      └──────────┘ └──────────┘└──────────┘     
staq
     ┌─────────────┐   ┌───┐    ┌───┐┌───┐┌───┐
q_0: ┤ U1(0.13231) ├───┤ S ├────┤ H ├┤ T ├┤ X ├
     └────┬───┬────┘┌──┴───┴───┐└───┘└───┘└─┬─┘
q_1: ─────┤ H ├─────┤ U1(3π/4) ├────────────■──
        

In [13]:
def count_gates(qasm, gate_names):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in gate_names:
            count += 1
    return count


def count_rot_z(qasm, theta):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name == 'rz'


def count_t_gates(qasm):
    return count_gates(qasm, ['t', 'tdag'])


def count_cnot_gates(qasm):
    return count_gates(qasm, ['cx'])


def count_total_gates(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    return len(list(qc))


ImportError: cannot import name 'SolovayKitaevDecomposition' from 'qiskit.transpiler.passes' (/Users/adrianlehmann/Code/CMSC32900-project-quantum-compiler-benchmark/quantum-compiler-benchmark/venv/lib/python3.9/site-packages/qiskit/transpiler/passes/__init__.py)

In [37]:
import time

total_time = dict()
total_gates = dict()
total_cnot = dict()
total_t_gates = dict()

# Initalize
for compiler_name_1 in compilers:
    for compiler_name_2 in compilers:
        total_time[compiler_name_1 + compiler_name_2] = 0
        total_gates[compiler_name_1 + compiler_name_2] = 0
        total_cnot[compiler_name_1 + compiler_name_2] = 0
        total_t_gates[compiler_name_1 + compiler_name_2] = 0

# Benchmark individually
for benchmark_file in benchmark_files:
    print(benchmark_file)
    fh = open(benchmark_file, "r")
    benchmark = fh.read()
    fh.close()
    assert not benchmark == ""
    for compiler_name_1 in compilers:
        for compiler_name_2 in compilers:
            print(compiler_name_1)
            print(compiler_name_2)
            start = time.perf_counter()
            opt_qasm = compilers[compiler_name_1](benchmark)
            if compiler_name_1 != compiler_name_2:
                opt_qasm = compilers[compiler_name_2](benchmark)
            stop = time.perf_counter()
            total_time[compiler_name_1 + compiler_name_2] += (stop - start)
            total_gates[compiler_name_1 + compiler_name_2] += count_total_gates(opt_qasm)
            total_cnot[compiler_name_1 + compiler_name_2] += count_cnot_gates(opt_qasm)
            total_t_gates[compiler_name_1 + compiler_name_2] += count_t_gates(opt_qasm)
print(total_time)

Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
qiskit


QasmError: "Cannot find gate definition for 'ccz', line 16 file "